In [1]:
## On Not being able to use python for Despina's. She said in L10 that the only thing she wants to see is that we've done it and that it wont be marked.
## Only assessed on whether it works or not

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from datetime import datetime, time 

## Importing hand-made modules
import sys
scraper_modules_path = 'C:/Users/ABour/Desktop/Energy Systems & Data Analytics/Modules/Energy Analysis - Aidan/Smart Meters/custom_modules'
if scraper_modules_path not in sys.path:
    sys.path.append(scraper_modules_path)

import pre_processing as pp
import feature_gen as fg

<br>

### Reading in and time series feature generation

In [2]:
batch = 4
df_smart_meters = pd.read_pickle(f'data/batch_{batch}.pkl')

In [3]:
df_smart_meters_std = df_smart_meters.drop(columns='stdorToU').copy()

df_smart_meters_std

,id,datetime,kWh
24158,MAC000003,2012-02-20 13:00:00,0.976
24159,MAC000003,2012-02-20 13:30:00,0.834
24160,MAC000003,2012-02-20 14:00:00,0.577
24161,MAC000003,2012-02-20 14:30:00,0.63
24162,MAC000003,2012-02-20 15:00:00,0.679
24163,MAC000003,2012-02-20 15:30:00,0.321
24164,MAC000003,2012-02-20 16:00:00,0.28
24165,MAC000003,2012-02-20 16:30:00,0.202
24166,MAC000003,2012-02-20 17:00:00,0.187
24167,MAC000003,2012-02-20 17:30:00,0.187


In [4]:
print(f'Std Size: {df_smart_meters_std.shape[0]}')

Std Size: 33775684


In [5]:
def pre_proc_ts_feature_gen(df):
    df = pp.generate_ts_features_df(df)
    df['kWh'] = df.kWh.replace('Null', None).astype(float)
    return df

df_smart_meters_std = pre_proc_ts_feature_gen(df_smart_meters_std)

In [6]:
plot_households = False

if plot_households == True:
    for _id in df_smart_meters.id.unique():
        ## Filter df for smart meter id
        df_smart_meter = df_smart_meters[df_smart_meters.id==_id]

        ## Plots
        pp.plot_SP_mean(_id, df_smart_meter)
        pp.plot_datetime_heatmap(_id, df_smart_meter)
        pp.plot_daily_peak_dist(_id, df_smart_meter)
    
## Try dropping zero values before

<br>

### Feature Generation

<img src="img/SMAP features.png">

In [7]:
## Creating array of SMAP feature keywords

## Need to add a pre-pre-filter for night & weekend combo, five so far
## They have mean daily maximum usage as well as mean peak usage, are they not the same? 
## Need a method to filter out peak usage
## Need to add seasonal filters

SMAP_features = np.array([
    [None, None, None, 'mean'], # Mean usage
    [None, None, None, 'var'], # Variance of usage
    [None, 'DoY', 'max', 'mean'], # Mean peak usage
    [None, 'DoY', 'max', 'var'], # Variance of peak usage
    ['day', None, None, 'mean'], # Mean night usage
    ['night', None, None, 'mean'], # Mean day usage
    ['day', None, None, 'var'], # Variance of night usage
    ['weekend', None, None, 'mean'], # Mean weekday usage
    ['weekend', None, None, 'var'], # Variance of weekday usage
    [None, 'DoY', 'min', 'mean'], # Mean trough usage
    [None, 'SP', 'var', 'mean'], # Mean of half-hourly coefficients of variation
    ['not_lunch', None, None, 'mean'], # Mean lunchtime usage
    [None, None, None, 'max'], # Peak usage
    [None, None, None, 'min'], # Trough usage
    ['night', None, None, 'var'], # Variance of day usage
    ['weekend', 'DoY', 'max', 'mean'], # Mean weekday peak usage
    ['weekend', 'DoY', 'max', 'var'], # Variance of weekday peak usage
    ['weekday', None, None, 'mean'], # Mean weekend usage
    ['weekday', None, None, 'var'], # Variance of weekend usage
    [None, 'SP', 'mean', None], # Mean of half-hourly usage
    [None, 'SP', 'var', None], # Variance of half-hourly usage
])

In [8]:
def generate_household_features(df_smart_meters, SMAP_features):
    house_dict = dict()

    for _id in df_smart_meters.id.unique():
        house_dict[_id] = dict() # Creating dictionary entry for new household id
        df_smart_meter = df_smart_meters[df_smart_meters.id==_id] # Filter df for smart meter id

        for i in range(SMAP_features.shape[0]):
            drop, time_step, time_step_feature, final_feature = SMAP_features[i]
            house_dict = fg.feature_gen(df_smart_meter, _id, house_dict, drop=drop, time_step=time_step, time_step_feature=time_step_feature, final_feature=final_feature)

    return house_dict

std_household_dict = generate_household_features(df_smart_meters_std, SMAP_features)

In [9]:
def household_dict_2_df(house_dict):
    df_households = pd.DataFrame(house_dict).T
    df_households['day/night'] = df_households['drop_night_combined_mean']/df_households['drop_day_combined_mean']

    df_households.index.name = 'household_id'
    return df_households

df_std = household_dict_2_df(std_household_dict)

In [13]:
df_std.to_csv(f'data/std_ts_features_batch_{batch}.csv')

In [11]:
df_std

,DoY_drop_weekend_max_combined_mean,DoY_drop_weekend_max_combined_var,DoY_max_combined_mean,DoY_max_combined_var,DoY_min_combined_mean,SP_10_mean,SP_10_var,SP_11_mean,SP_11_var,SP_12_mean,...,drop_not_lunch_combined_mean,drop_weekday_combined_mean,drop_weekday_combined_var,drop_weekend_combined_mean,drop_weekend_combined_var,max,mean,min,var,day/night
household_id,,,,,,,,,,,,,,,,,,,,,
MAC000003,2.028474,0.617751,2.221005,0.645825,0.041082,0.688474,0.743648,0.720512,0.634325,0.644071,...,0.239122,0.391339,0.380096,0.400169,0.377912,3.921,0.397658,0.007,0.378538,0.353108
MAC000008,1.173538,0.231600,1.251699,0.266600,0.172186,0.223647,0.006200,0.223155,0.006514,0.229172,...,0.426213,0.360521,0.062979,0.364131,0.056585,3.581,0.363101,0.000,0.058411,1.230245
MAC000013,0.245340,0.004336,0.253937,0.003626,0.029598,0.079885,0.001316,0.078694,0.001286,0.078534,...,0.087318,0.109605,0.002674,0.110266,0.003396,0.505,0.110078,0.023,0.003190,0.864490
MAC000019,0.627383,0.100441,0.761926,0.067463,0.023541,0.063808,0.000667,0.064889,0.000682,0.064288,...,0.185269,0.134171,0.018081,0.127720,0.016018,1.641,0.129560,0.010,0.016614,1.599756
MAC000022,0.678617,0.069552,0.746858,0.060955,0.085536,0.177475,0.021246,0.173744,0.020846,0.172959,...,0.244984,0.261137,0.034192,0.261619,0.032590,1.449,0.261481,0.016,0.033046,0.839593
MAC000024,2.412186,1.065544,2.641216,0.985066,0.031189,0.244914,0.181920,0.230894,0.141837,0.217660,...,0.554358,0.415468,0.376096,0.441599,0.391775,5.025,0.434170,0.026,0.387447,0.877763
MAC000026,1.203298,0.293255,1.218148,0.284301,0.107549,0.154557,0.001445,0.153015,0.001542,0.153830,...,0.278341,0.181490,0.013967,0.290803,0.070399,2.856,0.259620,0.084,0.056736,1.350878
MAC000028,0.354791,0.050105,0.405333,0.044982,0.041582,0.076271,0.001613,0.075334,0.001562,0.076862,...,0.091411,0.096509,0.004729,0.101672,0.005452,1.657,0.100192,0.000,0.005250,1.098023
MAC000029,0.564836,0.209846,0.738872,0.269995,0.005202,0.047186,0.000887,0.048766,0.001149,0.055615,...,0.070036,0.074060,0.018864,0.069467,0.012796,2.103,0.070777,0.000,0.014530,1.406834
